In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
import random
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score


In [10]:
file = open('Labeled_and_unlabeled_Data_Iowa_Contracts/Labeled/merged_data_with_text.csv', 'r')
df = pd.read_csv(file)

df['Institutional_Form'] = df['Institutional_Form'].str.lower()


df['Institutional_Form'].value_counts()

Institutional_Form
service contract                       372
joint operations                       125
new joint entities                      81
resource sharing                        45
grants/other subsidies                  19
joint facilities/co-location            12
transfer of authority                   11
transfer of responsibility              11
cooperative purchasing arrangements      6
joint facilities                         5
joint production                         3
other                                    2
cooperative purchasing agreement         1
Name: count, dtype: int64

In [12]:
df[df['Institutional_Form'] == 'cooperative purchasing agreement']['text'].values


array(['28E Agreement Matt Schultz Secretary of State State of Iowa FOR OFFICE USE ONLY: PLEASE READ INSTRUCTIONS ON BACK BEFORE COMPLETING THIS FORM Item 3. The purpose of this agreement is: (please be specific) _________________________________________________________________________________ _________________________________________________________________________________ _________________________________________________________________________________ _________________________________________________________________________________ Item 6. Attach two copies of the agreement to this form if not filing online. Item 7. The primary contact for further information regarding this agreement is: (optional) LAST Name _________________________________ FIRST Name ___________________________ Title ________________________________________ Department ____________________________ Email ______________________________________ Phone_________________________________ Item 4. The duration of this agreem

In [9]:
file = open('pdf_ocr_dataset1.csv', 'r')
df = pd.read_csv(file)
df

,PDF_ID,Folder_Name,OCR_Text
0,L006807,Education_OCR,MUY\nCity of Manchester\n208 East Main St.\nFI...
1,M508110,Education_OCR,Vinton- Shell shu rg 28 E AGREEMENT Kirkwood C...
2,M501807,Education_OCR,D. \nExcept as otherwise provided in Paragraph...
3,M035242,Education_OCR,Section A. \nThe College agrees as follows: \n...
4,M016858,Education_OCR,175\nRED FOR RECORD THE\n11\nDAY OF\nSTATE OF ...
...,...,...,...
19531,M015522,Planning_OCR,"BOONE COUNTY \nIN WITNESS WHEREOF, Boone Count..."
19532,M032688,Planning_OCR,Planning and Zoning Resolution #2005-10 \nPage...
19533,M007240,Planning_OCR,"STORY COUNTY \nIN WITNESS WHEREOF, Story Count..."
19534,M506159,Planning_OCR,FILED Matt Schultz 28 E Secretary of State Agr...


In [162]:
#dropping the class with only one instance
class_counts = df['Institutional_Form'].value_counts()
classes_to_drop = class_counts[class_counts <= 1].index
df = df[~df['Institutional_Form'].isin(classes_to_drop)]

In [163]:
df

,ID,Service _Area,Length,Renewal,Purpose,Institutional_Form,Des_of_IM,Second_Institutional_Form,Des_of_SIM,Funding,...,city_count,county_count,fire_district_count,private_or_nonprofit_count,regional_or_district_count,school_district_count,state_agency_count,township_count,other_count,text
0,M500728,Criminal,Indefinite,No,"coordinate drug law investigation, enforcement...",joint operations,The participating members created a task force...,No,NaN,No,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,28E Agreement Michael A. Mauro Secretary of St...
1,M501039,Criminal,indefinite,No,Implement joint and/or cooeprative action in d...,joint operations,The participating members created a task force...,No,NaN,"Obtain local, state/federal grant monies vaila...",...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,28E Agreement Michael A. Mauro Secretary of St...
2,M501047,Criminal,5,No,The county (Polk) confers upon the City and th...,transfer of authority,The county authorize the city to procecute off...,No,NaN,No,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28E Agreement Michael A. Mauro Secretary of St...
3,M501251,Criminal,Indefinite,No,Creation of taskforce to Jointly combat the il...,new joint entities,A multi-juristiction task force was organized ...,No,NaN,Each agency must be responsible for the cost i...,...,8.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28E Agreement Michael A. Mauro Secretary of St...
4,M503033,Criminal,Indefinite,No,"Given conflice of interest, the chief judge of...",transfer of responsibility,One attorney in another county was authorized ...,No,NaN,No,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28E Agreement Michael A. Mauro Secretary of St...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,M508152,NaN,NaN,NaN,NaN,service contract,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28E Agreement Paul D. Pate Secretary of State ...
689,M508881,NaN,NaN,NaN,NaN,service contract,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28E Agreement Paul D. Pate Secretary of State ...
690,M508886,NaN,NaN,NaN,NaN,service contract,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28E Agreement Paul D. Pate Secretary of State ...
691,M508949,NaN,NaN,NaN,NaN,service contract,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28E Agreement Paul D. Pate Secretary of State ...


In [126]:
df.isna().sum()

ID                              0
Service _Area                  65
Length                        272
Renewal                       272
Purpose                       272
Institutional_Form              0
Des_of_IM                     272
Second_Institutional_Form     272
Des_of_SIM                    691
Funding                       272
Shared_Service                272
Des_of S_                     553
Effic                         272
Effec                         272
Quality                       272
city_count                    272
county_count                  272
fire_district_count           272
private_or_nonprofit_count    272
regional_or_district_count    272
school_district_count         272
state_agency_count            272
township_count                272
other_count                   272
text                            0
dtype: int64

**Training the model with the purpose column**

In [165]:
#SVM
df.dropna(subset=['Purpose', 'Institutional_Form'], inplace=True)
X = df['Purpose']
df['Institutional_Form'] = df['Institutional_Form'].str.lower()
y = df['Institutional_Form']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [132]:
y

0                joint operations
1                joint operations
2           transfer of authority
3              new joint entities
4      transfer of responsibility
                  ...            
415              service contract
416              service contract
417              service contract
418         transfer of authority
419              service contract
Name: Institutional_Form, Length: 420, dtype: object

In [164]:
# Convert the text data into TF-IDF features
tfidf = TfidfVectorizer(max_features=5000)  # max_features limits the number of features
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

AttributeError: 'numpy.int64' object has no attribute 'lower'

In [134]:

# Initialize the SVM model
svm = SVC(kernel='linear', random_state=42)

# Train the SVM model on the training set
svm.fit(X_train_tfidf, y_train)

# Predict the labels on the test set
y_pred = svm.predict(X_test_tfidf)

# Print the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Print the classification report
print('Classification Report:')
print(classification_report(y_test, y_pred))


Accuracy: 74.29%
Classification Report:
                            precision    recall  f1-score   support

    grants/other subsidies       1.00      0.33      0.50         3
          joint facilities       0.00      0.00      0.00         2
          joint operations       0.62      0.76      0.68        21
        new joint entities       0.75      1.00      0.86         6
          resource sharing       0.50      0.14      0.22         7
          service contract       0.79      0.92      0.85        59
     transfer of authority       0.00      0.00      0.00         2
transfer of responsibility       0.00      0.00      0.00         5

                  accuracy                           0.74       105
                 macro avg       0.46      0.39      0.39       105
              weighted avg       0.67      0.74      0.69       105



/opt/local/miniforge/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/local/miniforge/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/local/miniforge/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [166]:
#Encoding the labels from the Institutional form column

possible_labels = df.Institutional_Form.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict
df['label'] = df.Institutional_Form.replace(label_dict)


/var/folders/8q/rpgcbn4n62l344xqhclfyn4c0000gn/T/ipykernel_92665/950600512.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df.Institutional_Form.replace(label_dict)


In [167]:
#BERT
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['Institutional_Form', 'label', 'data_type']).count()

ID  Service _Area  \
Institutional_Form                  label data_type                       
cooperative purchasing arrangements 8     train        5              5   
                                          val          1              1   
grants/other subsidies              4     train       13             13   
                                          val          2              2   
joint facilities                    9     train        4              4   
                                          val          1              1   
joint operations                    0     train       65             65   
                                          val         12             12   
new joint entities                  2     train       51             51   
                                          val          9              9   
other                               7     train        2              2   
resource sharing                    6     train       20             20   
                                          val          3              3   
service contract                    5     train      179            179   
                                          val         32             32   
transfer of authority               1     train        9              9   
                                          val          1              1   
transfer of responsibility          3     train        9              9   
                                          val          2              2   

                                                     Length  Renewal  Purpose  \
Institutional_Form                  label data_type                             
cooperative purchasing arrangements 8     train           5        5        5   
                                          val             1        1        1   
grants/other subsidies              4     train          13       13       13   
                                          val             2        2        2   
joint facilities                    9     train           4        4        4   
                                          val             1        1        1   
joint operations                    0     train          65       65       65   
                                          val            12       12       12   
new joint entities                  2     train          51       51       51   
                                          val             9        9        9   
other                               7     train           2        2        2   
resource sharing                    6     train          20       20       20   
                                          val             3        3        3   
service contract                    5     train         179      179      179   
                                          val            32       32       32   
transfer of authority               1     train           9        9        9   
                                          val             1        1        1   
transfer of responsibility          3     train           9        9        9   
                                          val             2        2        2   

                                                     Des_of_IM  \
Institutional_Form                  label data_type              
cooperative purchasing arrangements 8     train              5   
                                          val                1   
grants/other subsidies              4     train             13   
                                          val                2   
joint facilities                    9     train              4   
                                          val                1   
joint operations                    0     train             65   
                                          val               12   
new joint entities                  2     train             51   
                                          val     

In [168]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Purpose.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Purpose.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

/opt/local/miniforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/local/miniforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `paddi

In [169]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [170]:
batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [171]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)


/opt/local/miniforge/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [172]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def f1_score_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        
        # Calculate F1 score for the current class
        f1 = f1_score(y_true, y_preds, average='macro')
        
        print(f'Class: {label_dict_inverse[label]}')
        print(f'F1 Score: {f1:.2f}\n')

In [173]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [174]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):
    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total / len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    loss_train_avg = loss_train_total / len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    
    # Print F1 score for each class
    print("\nF1 Score for each class:")
    f1_score_per_class(predictions, true_vals)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/119 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.6659159843160325
Validation loss: 1.1526185671488445
F1 Score (Weighted): 0.5944026733500417

F1 Score for each class:
Class: joint operations
F1 Score: 0.13

Class: transfer of authority
F1 Score: 0.00

Class: new joint entities
F1 Score: 0.47

Class: transfer of responsibility
F1 Score: 0.00

Class: grants/other subsidies
F1 Score: 0.00

Class: service contract
F1 Score: 1.00

Class: resource sharing
F1 Score: 0.00

Class: cooperative purchasing arrangements
F1 Score: 0.00

Class: joint facilities
F1 Score: 0.00



Epoch 2:   0%|          | 0/119 [00:00<?, ?it/s]


Epoch 2
Training loss: 1.1125905888170755
Validation loss: 1.119385428017094
F1 Score (Weighted): 0.5523809523809523

F1 Score for each class:
Class: joint operations
F1 Score: 0.10

Class: transfer of authority
F1 Score: 0.00

Class: new joint entities
F1 Score: 0.29

Class: transfer of responsibility
F1 Score: 0.00

Class: grants/other subsidies
F1 Score: 0.00

Class: service contract
F1 Score: 1.00

Class: resource sharing
F1 Score: 0.00

Class: cooperative purchasing arrangements
F1 Score: 0.00

Class: joint facilities
F1 Score: 0.00



...


Epoch 8
Training loss: 0.4311023123656251
Validation loss: 1.241504737176001
F1 Score (Weighted): 0.6987134502923976

F1 Score for each class:
Class: joint operations
F1 Score: 0.22

Class: transfer of authority
F1 Score: 0.00

Class: new joint entities
F1 Score: 0.29

Class: transfer of responsibility
F1 Score: 0.33

Class: grants/other subsidies
F1 Score: 0.00

Class: service contract
F1 Score: 1.00

Class: resource sharing
F1 Score: 0.25

Class: cooperative purchasing arrangements
F1 Score: 0.00

Class: joint facilities
F1 Score: 0.00



Epoch 9:   0%|          | 0/119 [00:00<?, ?it/s]


Epoch 9
Training loss: 0.4205875914982137
Validation loss: 1.2613215094343537
F1 Score (Weighted): 0.6987134502923976

F1 Score for each class:
Class: joint operations
F1 Score: 0.22

Class: transfer of authority
F1 Score: 0.00

Class: new joint entities
F1 Score: 0.29

Class: transfer of responsibility
F1 Score: 0.33

Class: grants/other subsidies
F1 Score: 0.00

Class: service contract
F1 Score: 1.00

Class: resource sharing
F1 Score: 0.25

Class: cooperative purchasing arrangements
F1 Score: 0.00

Class: joint facilities
F1 Score: 0.00



Epoch 10:   0%|          | 0/119 [00:00<?, ?it/s]


Epoch 10
Training loss: 0.39205652024565624
Validation loss: 1.2215843583412824
F1 Score (Weighted): 0.6908427224216698

F1 Score for each class:
Class: joint operations
F1 Score: 0.22

Class: transfer of authority
F1 Score: 0.00

Class: new joint entities
F1 Score: 0.29

Class: transfer of responsibility
F1 Score: 0.33

Class: grants/other subsidies
F1 Score: 0.00

Class: service contract
F1 Score: 0.49

Class: resource sharing
F1 Score: 0.25

Class: cooperative purchasing arrangements
F1 Score: 0.00

Class: joint facilities
F1 Score: 0.00



**Now training the model with the whole text**

In [140]:
#SVM
df.dropna(subset=['text', 'Institutional_Form'], inplace=True)
X = df['text']
df['Institutional_Form'] = df['Institutional_Form'].str.lower()
y = df['Institutional_Form']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Convert the text data into TF-IDF features
tfidf = TfidfVectorizer(max_features=5000)  # max_features limits the number of features
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Initialize the SVM model
svm = SVC(kernel='linear', random_state=42)

# Train the SVM model on the training set
svm.fit(X_train_tfidf, y_train)

# Predict the labels on the test set
y_pred = svm.predict(X_test_tfidf)

# Print the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Print the classification report
print('Classification Report:')
print(classification_report(y_test, y_pred))
# Initialize the SVM model
svm = SVC(kernel='linear', random_state=42)

# Train the SVM model on the training set
svm.fit(X_train_tfidf, y_train)

# Predict the labels on the test set
y_pred = svm.predict(X_test_tfidf)

# Print the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Print the classification report
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 67.79%
Classification Report:
                                     precision    recall  f1-score   support

cooperative purchasing arrangements       0.00      0.00      0.00         1
             grants/other subsidies       0.00      0.00      0.00         7
                   joint facilities       0.00      0.00      0.00         1
       joint facilities/co-location       0.00      0.00      0.00         2
                   joint operations       0.67      0.27      0.38        37
                 new joint entities       1.00      0.68      0.81        19
                   resource sharing       1.00      0.06      0.11        17
                   service contract       0.65      0.98      0.79       119
              transfer of authority       0.00      0.00      0.00         3
         transfer of responsibility       0.00      0.00      0.00         2

                           accuracy                           0.68       208
                          macro av

/opt/local/miniforge/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/local/miniforge/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/local/miniforge/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 67.79%
Classification Report:
                                     precision    recall  f1-score   support

cooperative purchasing arrangements       0.00      0.00      0.00         1
             grants/other subsidies       0.00      0.00      0.00         7
                   joint facilities       0.00      0.00      0.00         1
       joint facilities/co-location       0.00      0.00      0.00         2
                   joint operations       0.67      0.27      0.38        37
                 new joint entities       1.00      0.68      0.81        19
                   resource sharing       1.00      0.06      0.11        17
                   service contract       0.65      0.98      0.79       119
              transfer of authority       0.00      0.00      0.00         3
         transfer of responsibility       0.00      0.00      0.00         2

                           accuracy                           0.68       208
                          macro av

/opt/local/miniforge/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/local/miniforge/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/local/miniforge/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [147]:
#Encoding the labels from the Institutional form column
df['Institutional_Form'] = df['Institutional_Form'].str.lower()
possible_labels = df.Institutional_Form.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

df['label'] = df.Institutional_Form.replace(label_dict)


/var/folders/8q/rpgcbn4n62l344xqhclfyn4c0000gn/T/ipykernel_92665/2099948065.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df.Institutional_Form.replace(label_dict)


In [151]:
#BERT
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

#df.groupby(['Institutional_Form', 'label', 'data_type']).count()

In [158]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def f1_score_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        
        # Calculate F1 score for the current class
        f1 = f1_score(y_true, y_preds, average='macro')
        
        print(f'Class: {label_dict_inverse[label]}')
        print(f'F1 Score: {f1:.2f}\n')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different na

In [160]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):
    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total / len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    loss_train_avg = loss_train_total / len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    
    # Print F1 score for each class
    print("\nF1 Score for each class:")
    f1_score_per_class(predictions, true_vals)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/196 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.5083030358869203
Validation loss: 1.4808655159814017
F1 Score (Weighted): 0.37692307692307686

F1 Score for each class:
Class: joint operations
F1 Score: 0.00

Class: transfer of authority
F1 Score: 0.00

Class: new joint entities
F1 Score: 0.00

Class: transfer of responsibility
F1 Score: 0.00

Class: grants/other subsidies
F1 Score: 0.00

Class: service contract
F1 Score: 1.00

Class: resource sharing
F1 Score: 0.00

Class: cooperative purchasing arrangements
F1 Score: 0.00

Class: joint facilities
F1 Score: 0.00

Class: joint facilities/co-location
F1 Score: 0.00



Epoch 2:   0%|          | 0/196 [00:00<?, ?it/s]


Epoch 2
Training loss: 1.534452207690599
Validation loss: 1.4826620050839014
F1 Score (Weighted): 0.37692307692307686

F1 Score for each class:
Class: joint operations
F1 Score: 0.00

Class: transfer of authority
F1 Score: 0.00

Class: new joint entities
F1 Score: 0.00

Class: transfer of responsibility
F1 Score: 0.00

Class: grants/other subsidies
F1 Score: 0.00

Class: service contract
F1 Score: 1.00

Class: resource sharing
F1 Score: 0.00

Class: cooperative purchasing arrangements
F1 Score: 0.00

Class: joint facilities
F1 Score: 0.00

Class: joint facilities/co-location
F1 Score: 0.00



...


Epoch 8
Training loss: 1.4935325809887476
Validation loss: 1.4785712701933724
F1 Score (Weighted): 0.37692307692307686

F1 Score for each class:
Class: joint operations
F1 Score: 0.00

Class: transfer of authority
F1 Score: 0.00

Class: new joint entities
F1 Score: 0.00

Class: transfer of responsibility
F1 Score: 0.00

Class: grants/other subsidies
F1 Score: 0.00

Class: service contract
F1 Score: 1.00

Class: resource sharing
F1 Score: 0.00

Class: cooperative purchasing arrangements
F1 Score: 0.00

Class: joint facilities
F1 Score: 0.00

Class: joint facilities/co-location
F1 Score: 0.00



Epoch 9:   0%|          | 0/196 [00:00<?, ?it/s]


Epoch 9
Training loss: 1.487619544474446
Validation loss: 1.4789154052734375
F1 Score (Weighted): 0.37692307692307686

F1 Score for each class:
Class: joint operations
F1 Score: 0.00

Class: transfer of authority
F1 Score: 0.00

Class: new joint entities
F1 Score: 0.00

Class: transfer of responsibility
F1 Score: 0.00

Class: grants/other subsidies
F1 Score: 0.00

Class: service contract
F1 Score: 1.00

Class: resource sharing
F1 Score: 0.00

Class: cooperative purchasing arrangements
F1 Score: 0.00

Class: joint facilities
F1 Score: 0.00

Class: joint facilities/co-location
F1 Score: 0.00



Epoch 10:   0%|          | 0/196 [00:00<?, ?it/s]


Epoch 10
Training loss: 1.4973467147471953
Validation loss: 1.4789154052734375
F1 Score (Weighted): 0.37692307692307686

F1 Score for each class:
Class: joint operations
F1 Score: 0.00

Class: transfer of authority
F1 Score: 0.00

Class: new joint entities
F1 Score: 0.00

Class: transfer of responsibility
F1 Score: 0.00

Class: grants/other subsidies
F1 Score: 0.00

Class: service contract
F1 Score: 1.00

Class: resource sharing
F1 Score: 0.00

Class: cooperative purchasing arrangements
F1 Score: 0.00

Class: joint facilities
F1 Score: 0.00

Class: joint facilities/co-location
F1 Score: 0.00

